In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import scale
from scipy.cluster.hierarchy import linkage, leaves_list

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

pd.set_option('display.max_columns', 500)

In [3]:
mapp     = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t', index_col='ensembl')
bspheno  = pd.read_csv('../data/BrainSpan/columns_metadata.csv', sep='\t')
bsrows   = pd.read_csv('../data/BrainSpan/rows_metadata.csv', sep=',')
rpkm     = pd.read_csv('../data/BrainSpan/expression_matrix.csv', sep=',', header=None)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
rpkm.index = bsrows.ensembl_gene_id
rpkm = rpkm.drop(0, axis=1)

bspheno = bspheno.set_index('column_num', drop=False)

# define minor epochs 

8≤ Age <10 PCW Early fetal 

10≤ Age <13 PCW Early fetal 

13≤ Age <16 PCW Early mid-fetal 

16≤ Age <19 PCW Early mid-fetal 

24≤ Age <38 PCW Late fetal 

Birth≤ Age <6 Postnatal months (M) Neonatal and early infancy 

6 M≤ Age <1 Year (Y) Late infancy 

1≤ Age <6 Y Early childhood 

6≤ Age <12 Y Middle and late childhood 

12≤ Age <20 Y Adolescence 

20≤ Age <40 Y Young adulthood 

40≤ Age <60 Y Middle adulthood

In [5]:
def to_minor_epoch(age):
    if age.endswith('pcw'):
        s = age[:-4]
        s = int(s)
        if 8 <= s and s < 10:
            return 'early fetal'
        if 10 <= s and s < 13:
            return 'early fetal'
        if 13 <= s and s < 16:
            return 'early mid-fetal'
        if 16 <= s and s < 19:
            return 'early mid-fetal'
        if 19 <= s and s < 24:
            return 'late mid-fetal'
        if 24 <= s and s < 38:
            return 'late fetal'
    if age.endswith('mos'):
        s = age[:-4]
        s = int(s)
        if s < 6:
            return 'early infancy'
        if 6 <= s and s < 12:
            return 'late infancy'
    if age.endswith('yrs'):
        s = age[:-4]
        s = int(s)
        if 1 <= s and s < 6:
            return 'early childhood'
        if 6 <= s and s < 12:
            return 'middle and late childhood'
        if 12 <= s and s < 20:
            return 'adolescence'
        if 20 <= s and s < 40:
            return 'young adulthood'
        if 40 <= s and s < 60:
            return 'middle adulthood'
    return age

bspheno['minor epoch'] = bspheno.age.apply(lambda s: to_minor_epoch(s))

In [6]:
adult_fetal_region_map = {
    'cerebellum': [ 'CB', 'URL', 'CBC' ],
    'hippocampus': ['HIP'],
    'amygdala': ['AMY'],
    'striatum': ['STR','MGE', 'LGE', 'CGE'],
    'thalamus': ['DTH', 'MD'],
    'medial frontal cortex': ['MFC'],
    'orbitofrontal cortex': ['OFC'],
    'dorsolateral frontal cortex' : ['DFC'],
    'ventrolateral frontal cortex' : ['VFC'],
    'primary motor cortex': ['M1C', 'M1C-S1C', 'PCx'],
    'primary somatosensory cortex': ['S1C', 'M1C-S1C', 'PCx'],
    'inferior parietal cortex': ['IPC', 'PCx'],
    'primary auditory cortex': ['A1C', 'TCx'],
    'superior temporal cortex': ['STC', 'TCx'],
    'inferior temporal cortex': ['ITC', 'TCx'],
    'primary visual cortex': ['V1C', 'Ocx'],   
}

In [7]:
periods = list(bspheno['minor epoch'].unique())

df = rpkm.copy()
df = df[[]]
for t in adult_fetal_region_map:
    for s in periods:
        df['{} {}'.format(t, s)] = '.'

In [8]:
for tissue in adult_fetal_region_map:
    for period in periods:
        samples = bspheno[(bspheno['minor epoch'] == period) & (bspheno.structure_acronym.isin(adult_fetal_region_map[tissue]))].index
        tp_rpkm = rpkm[samples]
        print(tissue, period, tp_rpkm.shape[1])
        df['{} {}'.format(tissue, period)] = tp_rpkm.T.median()

cerebellum early fetal 4
cerebellum early mid-fetal 4
cerebellum late mid-fetal 2
cerebellum late fetal 3
cerebellum early infancy 2
cerebellum late infancy 1
cerebellum early childhood 5
cerebellum middle and late childhood 3
cerebellum adolescence 4
cerebellum young adulthood 5
cerebellum middle adulthood 1
hippocampus early fetal 5
hippocampus early mid-fetal 6
hippocampus late mid-fetal 2
hippocampus late fetal 2
hippocampus early infancy 2
hippocampus late infancy 0
hippocampus early childhood 3
hippocampus middle and late childhood 3
hippocampus adolescence 3
hippocampus young adulthood 5
hippocampus middle adulthood 1
amygdala early fetal 5
amygdala early mid-fetal 6
amygdala late mid-fetal 1
amygdala late fetal 2
amygdala early infancy 3
amygdala late infancy 0
amygdala early childhood 4
amygdala middle and late childhood 3
amygdala adolescence 3
amygdala young adulthood 5
amygdala middle adulthood 1
striatum early fetal 9
striatum early mid-fetal 7
striatum late mid-fetal 2
st

In [9]:
summ = pd.DataFrame(index=rpkm.index)
summ['median'] = 0
summ['mean'] = 0
summ['std'] = 0

for gene in summ.index:
    summ.loc[gene, 'median'] = df.loc[gene].median()
    summ.loc[gene, 'mean'] = df.loc[gene].mean()
    summ.loc[gene, 'std'] = df.loc[gene].std()

In [10]:
def is_enriched_in_tissue(tpm, std_gene, med_gene):
    if tpm > med_gene + 2* std_gene:
#         print(tpm, std_gene, med_gene)
        return True
    return False


summ['enriched_tissues'] = '.'

for gene in summ.index:
    std_gene = summ.loc[gene, 'std']
    med_gene = summ.loc[gene, 'median']
    enrichements =  df.loc[gene].apply(is_enriched_in_tissue, std_gene=std_gene, med_gene=med_gene)
    enrichements = enrichements[enrichements].index.tolist()
#     print(gene, std_gene, med_gene, len(enrichements))

    enrichements = ';'.join(enrichements)
    summ.at[gene, 'enriched_tissues'] = enrichements

In [14]:
summ.index.name = 'ensembl'

In [15]:
summ.to_csv('brainspan_preferential_tissue_enrichment.tsv', sep='\t', index=True)

In [14]:
df2 = df.copy()
for col in df2.columns:
    df2[col] = 0

In [15]:
for gene in summ.index[:]:
    tissues = summ.at[gene, 'enriched_tissues']
    tissues = tissues.split(',')
    for tissue in tissues:
        df2.at[gene, tissue] = 1


In [16]:
df2.index.Name = 'ensembl'

In [18]:
df2 = df2.drop('', axis=1)

In [19]:
df2.to_csv('brainspan_preferential_tissue_expression_minor_epoch.tsv', sep='\t')